In [1]:
pip install evaluate --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import numpy as np
import evaluate
import torch

# Dataset uploading

In [4]:
"""
Note: Did not test on kaggle yet.
"""
import os 

DATASET_PATHS = {
    "local": {
        "train": "../../datasets/train_set.csv",
        "test": "../../datasets/test_set.csv"
    },
    "kaggle": {
        "train": "/kaggle/input/python-codes-time-complexity/train_set.csv",
        "test": "/kaggle/input/python-codes-time-complexity/test_set.csv"
    }
}

def upload_datasets(dataset_paths=DATASET_PATHS):
    for path in dataset_paths:
        if os.path.exists(dataset_paths[path]['train']) and os.path.exists(dataset_paths[path]['test']):
            return dataset_paths[path]['train'], dataset_paths[path]['test']

    return FileNotFoundError(f"Datasets do not exist in the current paths: {dataset_paths}")
            

train_set_path, test_set_path = upload_datasets()

# Metrics

## Hierarchy Complexity Score

### Ordering labels by Hierarchy

In [5]:
LABELS_HIERARCHY = {
    'constant': 1,
    'logn': 2,
    'linear': 3,
    'nlogn': 4,
    'quadratic': 5,
    'cubic': 6,
    'np': 7
}

N_CLASSES = len(LABELS_HIERARCHY)

### Writing the custom metric hc_score

In [6]:
def hc_score(y_true, y_pred, n_classes=N_CLASSES):
    assert len(y_true) == len(y_pred), f"The amount of y_true labels: {len(y_true)} does not equal to the amount of y_pred: {len(y_pred)}."

    y_pred = np.array([LABELS_HIERARCHY[pred] for pred in y_pred])
    y_true = np.array([LABELS_HIERARCHY[true] for true in y_true])
    n_samples = len(y_true)
    
    return (np.sum(np.abs(y_pred - y_true)) / n_classes) / n_samples

hierarchy_score = make_scorer(hc_score)

### All metrics

In [7]:
metrics = {'accuracy': 'accuracy', 'f1_macro': 'f1_macro', 'hierarchy_score': hc_score}

# Dataset uploading

In [8]:
train_set = load_dataset("csv", data_files=train_set_path)
test_set = load_dataset("csv", data_files=test_set_path)

train_labels = train_set['train']['complexity']
test_labels = test_set['train']['complexity']

# Checkpoint

In [9]:
checkpoints = ["microsoft/codebert-base", "neulab/codebert-python", "microsoft/graphcodebert-base", 
              "Salesforce/codet5-base", "Salesforce/codet5-base-codexglue-sum-python", "Salesforce/codet5p-220m-py",
              "Salesforce/codet5-base-multi-sum", "microsoft/unixcoder-base"]

# Tokenizing

## Label tokenizing

In [10]:
labelEncoder = LabelEncoder()
labelEncoder.fit(train_labels)

LabelEncoder()

## Feature tokenizing

In [11]:
def tokenize_data(samples, tokenizer):
    tokenized = tokenizer(samples['code'], truncation=True, max_length=512)
    tokenized['labels'] = labelEncoder.transform(samples['complexity'])
    return tokenized


def set_tokenizer(checkpoint):
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    X_train = train_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)
    X_eval = test_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)

    # Collator for batch padding
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    return tokenizer, data_collator, X_train, X_eval

# Model

In [12]:
def set_model(checkpoint, lora=None):
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7) if not lora else lora
    return model

# Hyperparameters

In [13]:
def set_training_args(checkpoint, batch_size=16):
    training_args = TrainingArguments(output_dir=f"training_results/{checkpoint}/", 
                                      eval_strategy="epoch",
                                      save_strategy="epoch",
                                      bf16=True,
                                      report_to='none',
                                      num_train_epochs=3, 
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size,
                                      gradient_accumulation_steps = 4,
                                      logging_steps=100,
                                      load_best_model_at_end=True,
                                      #label_names=['complexity']
                                     )
    return training_args

# Evaluating

In [14]:
def compute_metrics(eval_preds):
    labels = eval_preds.labels
    preds = eval_preds.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    # Calculate F-1 Macro
    f1_macro_score = f1(labels, preds)
    # Calculate Hierarchy Score
    hc_score = hierarchy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro_score,
        "hierarchy_score": hc_score
    }

# LoRA

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, 
    lora_alpha=16,
    target_modules = ["q_proj", "v_proj"] # Not sure about this
    lora_dropout=0.1,
    bias='none',
    modules_to_save=['classifier'] # Not sure about this one either
)

model_lora = get_peft_model(model=set_model("Salesforce/codet5p-770m-py"), peft_config=config)
model_lora.print_trainable_parameters()

# Trainer 

In [15]:
def train(checkpoint):
    tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)
    model = set_model(checkpoint)
    training_args = set_training_args(checkpoint=checkpoint, batch_size=2)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set['train'],
    eval_dataset=eval_set['train'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)
    
    trainer.train()


train(checkpoints[0])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 6.05 GB, other allocations: 11.97 GB, max allowed: 18.13 GB). Tried to allocate 147.26 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

# Flushing CUDA

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           

# Inference

In [ ]:
print(torch.cuda.device_count())

In [ ]:
device = torch.cuda.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def predict(inputs):
    # Tokenizing inputs
    test_sample = tokenizer(inputs, return_tensors='pt')
    inputs = Dataset.from_dict({key: value.to(model.device) for key, value in test_sample.items()})

    # Predicting & decoding inputs
    preds = trainer.predict(test_dataset=inputs)
    preds = labelEncoder.inverse_transform(y=np.ravel(np.argmax(preds.predictions, axis=-1)))
    
    return preds

In [ ]:
test_sample = """
class Solution:
    def isValid(self, s: str) -> bool:
        bracketMap = {"(": ")", "[": "]", "{": "}"}
        openSet = set(["(", "[", "{"])
        stack = []
        for char in s:
            if char in openSet:
                stack.append(char)
            elif stack and char == bracketMap[stack[-1]]:
                stack.pop()
            else:
                return False
        return stack == []
        """

predict(test_sample)